In [2]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../")

import torch
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import chained_seq2seq
from src.utils import STTDataModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Instantiate Model Class

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
dm = STTDataModule()
dm.setup("fit")
dm.setup("test")

train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

In [5]:
logger = TensorBoardLogger("../model/checkpoints/tb_logs", name="chained_seq2seq", log_graph=False)
model = chained_seq2seq()

In [6]:
seed_everything(42)
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    logger=logger,
    devices=1 if torch.cuda.is_available() else None,
    default_root_dir="../model/checkpoints/"
)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, datamodule=dm, ckpt_path="../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints/epoch=0-step=1815.ckpt")

Restoring states from the checkpoint path at ../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints/epoch=0-step=1815.ckpt
/home/anand/Development/tinkering/stt_postprocess/venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:345: UserWarning: The dirpath has changed from '../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints' to '../model/checkpoints/tb_logs/chained_seq2seq/version_6/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | encoder   | MarianMTModel    | 79.0 M
1 | decoder   | MarianMTModel    | 79.0 M
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
156 M     Trainable params
1.0 M     Non-trainable params

Sanity Checking: 0it [00:00, ?it/s]

Training: 1815it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
